# Apportionment Practice
We are setting up code to automate the apportionment process. Apportionment uses te hunting-hill method to assign house seats by looking at the geometric mean.
This code will do the math for our current 435 seats, show what that looks like with double the seats, or if we followed some verison of the wyoming rule.
A second portio of this code will then check to see what happens to election results over the last 20 years with the above changes, if we add DC or Puerto Rico, and if we proportionaly assign ec votes.

In [4]:
#setting up my packages
#the pandas package gives me data tools in a similar manner as tidyverse
import pandas as pd
#numpy expands these features futher allowing me to work more easily with series--i.e. vectors
import numpy as np
#plotly allows for greater and better graphing abilities
import plotly.express as px

In [5]:
#setting up my dataframes
census_data = pd.read_excel("Data/nst-est2019-01.xlsx", engine="openpyxl")


In [6]:
# Variables
#DC and Puerto Rico are not currently us states. We want to have this variable here so we can add them in as states later, should a user desire
excluded_states = [
    "district of columbia",
    "puerto rico"
]

#There is currently a cap of 435 house of rep members as laid out by statue in congress. A simple majority in both house could change how many seats are apportioned or in what manner. Total seats lets us show some options
total_seats = 435
#if we were to go with the wyoming rule we would divde the entire us population by the size of the smallest district (currently Wyoming) and that resulting number would be the number of house seats. It would still follow the Hunting-Hill method. as this is a floating variable we will do the math later, but a place holder int will be entered here
wyoming_rule_2010 = 547
#going by the principles of the federalist paper no. 55 there should be many more seats in congress. This will let us set the average district size to the mean over the history of the house.
federalist_55 = False
#these are the historical disctrict sizes placed into a series
district_sizes = pd.Series([34436, 34609, 36377, 42124, 49712, 71338, 93020, 122614, 130533, 151912, 173901, 193167, 210583, 243728, 280675, 301164, 334587, 410481, 469088, 510818, 571477, 646946, 709760])
population_2010 = np.sum(census_data["Census"])


In [7]:
#checks the first 5 rows of the orginal data to see what were working with
census_data.head()

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,.Alabama,4779736.0,4780125.0,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0
1,.Alaska,710231.0,710249.0,713910.0,722128.0,730443.0,737068.0,736283.0,737498.0,741456.0,739700.0,735139.0,731545.0
2,.Arizona,6392017.0,6392288.0,6407172.0,6472643.0,6554978.0,6632764.0,6730413.0,6829676.0,6941072.0,7044008.0,7158024.0,7278717.0
3,.Arkansas,2915918.0,2916031.0,2921964.0,2940667.0,2952164.0,2959400.0,2967392.0,2978048.0,2989918.0,3001345.0,3009733.0,3017804.0
4,.California,37253956.0,37254519.0,37319502.0,37638369.0,37948800.0,38260787.0,38596972.0,38918045.0,39167117.0,39358497.0,39461588.0,39512223.0


In [8]:
# rename columns
census_data = census_data.rename(columns={"Geographic Area": "state_name", "Census": "census_2010", 2019: "estimate_2019"})
# keep columns we need
census_data = census_data.reindex(columns=["state_name", "census_2010", "estimate_2019"])

In [9]:
# clean data
census_data["state_name"] = census_data.state_name.str.lower().str.replace(".", "")
census_data = census_data[census_data["state_name"].notnull()]

In [10]:
# set index
census_data = census_data.set_index("state_name")

In [11]:
# set statehood
census_data["statehood"] = True
census_data.loc[excluded_states, "statehood"] = False

In [12]:
census_data.head(9)

,census_2010,estimate_2019,statehood
state_name,,,
alabama,4779736.0,4903185.0,True
alaska,710231.0,731545.0,True
arizona,6392017.0,7278717.0,True
arkansas,2915918.0,3017804.0,True
california,37253956.0,39512223.0,True
colorado,5029196.0,5758736.0,True
connecticut,3574097.0,3565287.0,True
delaware,897934.0,973764.0,True
district of columbia,601723.0,705749.0,False


In [13]:
population_column = "census_2010"

# initial seats for the 2010 census populations
#checks to make sure we are in fact using the huntington-hill method and also removes those geographic areas which are not states
if not federalist_55:
    census_data["number_seats"] = np.where(census_data["statehood"], 1, 0)
#we now add house seats per the hunting-hill method which uses a geometric mean to figure out which added representative will have the greatest impact in lowering the the difference between all states.
priority_number_history = []
while census_data.number_seats.sum() < total_seats:
    priority_number = (
        census_data[population_column]
        / np.sqrt(census_data.number_seats * (census_data.number_seats + 1))
    )
    # remove excluded states
    priority_number = priority_number[priority_number != np.inf]
    
    priority_number_history.append(priority_number)
    
    largest_priority_number = priority_number.sort_values(ascending=False)[:1].index
    
    census_data.loc[largest_priority_number, "number_seats"] += 1

In [14]:
census_data.sort_values("number_seats", ascending=False).head()

,census_2010,estimate_2019,statehood,number_seats
state_name,,,,
california,37253956.0,39512223.0,True,53
texas,25145561.0,28995881.0,True,36
florida,18801310.0,21477737.0,True,27
new york,19378102.0,19453561.0,True,27
pennsylvania,12702379.0,12801989.0,True,18


In [15]:
census_data.sort_values("number_seats", ascending=True).head()

,census_2010,estimate_2019,statehood,number_seats
state_name,,,,
puerto rico,3725789.0,3193694.0,False,0
district of columbia,601723.0,705749.0,False,0
montana,989415.0,1068778.0,True,1
vermont,625741.0,623989.0,True,1
wyoming,563626.0,578759.0,True,1


In [16]:
#federalist 55 we are creating as many districts as there would be in the average district size had remained unchanged
average_district = np.mean(district_sizes)
federalist_districts = population_2010/average_district
print(federalist_districts)

1234.2055316372005


In [25]:
#The Real wyoming rule math
base_pop_2010 = census_data["census_2010"].min()
wyoming_rule = round(population_2010/base_pop_2010, 0)
print(wyoming_rule)

554.0


In [36]:
census_2010.head()


,state_name,census_2010,house_seats
0,.Alabama,4779736.0,1
1,.Alaska,710231.0,1
2,.Arizona,6392017.0,1
3,.Arkansas,2915918.0,1
4,.California,37253956.0,1
